### Brief Summary of Code
The code below was used to clean three csv files - a pre-program survey, a post-program survey, and normal CRM data pertaining to students that participated in programs offered by a fake study abroad provider - SAO. 
All data is masked as it originated from a real study abroad company. The code masks this data by creating fake studentid numbers, reduces the number of participants by a random percentage between 50% and 90%, and randomizes the cities in which they studied abroad. The data is cleaned and prepared for a left join in Tableau. 

In [ ]:
#Import programming languages for data cleaning/manipulation, as well as exporting to an excel file with 
#multiple sheets for Tableau - I always import matplotlib in case I want to include data visualizations in 
#my notebooks
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib as plt
import xlsxwriter
import random

In [ ]:
#Import datatime function in order to label records and exported files with today's date
import datetime as tdy     
datestr = tdy.datetime.today
tdy.datetime.today().strftime("%y%m%d")

In [ ]:
#upload first of three CSV files containing answers to surveys administered to clients during
#three stages of their product cycle - pre, mid, and post. I needed to use different encoding option
#because of the file format, most likely due to certain characters in the strings: ñ, í, é, á
df_PPS = pd.read_csv('Post-Program Survey.csv', encoding = "ISO-8859-1", low_memory=False)

In [ ]:
#For the purposes of this demonstration, we are only going to keep NPS score questions, along with outcomes assessment
#questions, related to skills gained during program participation, and specific learning outcomes questions, because
#of this, we can drop a number of columns present in the original survey exported csv, using the .drop function,
#starting with later columns-
df_PPS.drop(df_PPS.columns[180:185], axis=1, inplace=True)
df_PPS.drop(df_PPS.columns[68:165], axis=1, inplace=True)
df_PPS.drop(df_PPS.columns[15:56], axis=1, inplace=True)
df_PPS.drop(df_PPS.columns[14], axis=1, inplace=True)
df_PPS.drop(df_PPS.columns[11:13], axis=1, inplace=True)
df_PPS.drop(df_PPS.columns[10], axis=1, inplace=True)
df_PPS.drop(df_PPS.columns[6:9], axis=1, inplace=True)


In [ ]:
#change column headers for Post Program Survey, this will make it easier to create visualizations in Tableau
df_PPS.rename(columns={'Respondent ID':'respondent_ID', 
                       'Collector ID':'collector_ID',
                      'Start Date':'resp_start_date',
                       'End Date':'resp_end_date',
                       'IP Address':'IP_address',
                       'Email Address':'email',
                       'Custom Data 2':'program_code',
                       df_PPS.columns[7]:'Q1_would_rec',
                       df_PPS.columns[8]:'Q8_a_PG_dev_new_skills',
                       df_PPS.columns[9]:'Q8_b_PG_imprv_frgn_lang_skls',
                       df_PPS.columns[10]:'Q8_c_PG_learn_culture_way_of_life',
                       df_PPS.columns[11]:'Q8_d_PG_learn_history_host_country',
                       df_PPS.columns[12]:'Q8_e_PG_expl_heritage',
                       df_PPS.columns[13]:'Q8_f_PG_better_know_regional_polit_curnt_evnts_',
                       df_PPS.columns[14]:'Q8_g_PG_learned_about_myself',
                       df_PPS.columns[15]:'Q8_h_PG_earnd_credt_to_grad',
                       df_PPS.columns[16]:'Q8_i_PG_gained_independ',
                       df_PPS.columns[17]:'Q8_j_PG_expl_environ',
                       df_PPS.columns[18]:'Q8_k_PG_positiv_contribut_host_commun',
                       df_PPS.columns[19]:'Q8_l_PG_qual_other',
                       df_PPS.columns[20]:'Q32_a_DM_ID_Q1',
                       df_PPS.columns[21]:'Q32_b_DM_HI_Q1',
                       df_PPS.columns[22]:'Q32_c_DM_SP_Q1',
                       df_PPS.columns[23]:'Q32_d_DM_PF_Q1',
                       df_PPS.columns[24]:'Q32_e_DM_EV_Q1',
                       df_PPS.columns[25]:'Q33_a_DM_ID_Q2',
                       df_PPS.columns[26]:'Q33_b_DM_HI_Q2',
                       df_PPS.columns[27]:'Q33_c_DM_SP_Q2',
                       df_PPS.columns[28]:'Q33_d_DM_PF_Q2',
                       df_PPS.columns[29]:'Q33_e_DM_EV_Q2',
                       df_PPS.columns[30]:'Q34_a_DM_ID_Q3',
                       df_PPS.columns[31]:'Q34_b_DM_HI_Q3',
                       df_PPS.columns[32]:'Q34_c_DM_SP_Q3',
                       df_PPS.columns[33]:'Q34_d_DM_PF_Q3',
                       df_PPS.columns[34]:'Q34_e_DM_EV_Q3',
                      
}
                     , inplace=True)

In [ ]:
#Lets check out the dataframe to make sure it is set up the way we want. We still need to change datatypes, we can
#take care of that later, lets get the pre-departure survey sorted out now. 
df_PPS.info()

In [ ]:
#uploading the pre-departure survey, again using sepecific encoding. 
df_PDS = pd.read_csv('Pre-Departure Survey.csv', encoding = "ISO-8859-1", low_memory=False)

In [ ]:
#For the purposes of this demonstration, we are only going to keep NPS score questions, along with outcomes assessment
#questions, related to skills they hope to gain with program participation, and specific learning outcomes questions, because
#of this, we can drop a number of columns present in the original survey exported csv, using the .drop function,
#starting with later columns-
df_PDS.drop(df_PDS.columns[109:114], axis=1, inplace = True)
df_PDS.drop(df_PDS.columns[41:94], axis=1, inplace = True)
df_PDS.drop(df_PDS.columns[15:29], axis=1, inplace=True)
df_PDS.drop(df_PDS.columns[10:14], axis=1, inplace=True)
df_PDS.drop(df_PDS.columns[6:9], axis=1, inplace=True)

In [ ]:
#change column headers for Pre-departure survey dataframe, again making it easier to sort data in Tableau
df_PDS.rename(columns={'Respondent ID':'respondent_ID', 
                       'Collector ID':'collector_ID',
                      'Start Date':'resp_start_date',
                       'End Date':'resp_end_date',
                       'IP Address':'IP_address',
                       'Email Address':'email',
                       'First Name':'first_name',
                       'Last Name':'last_name',
                       'Custom Data 2':'program_code',
                       df_PDS.columns[7]:'Q1_would_rec',
                       df_PDS.columns[8]:'Q5_a_PG_dev_new_skills',
                       df_PDS.columns[9]:'Q5_b_PG_imprv_frgn_lang_skls',
                       df_PDS.columns[10]:'Q5_c_PG_learn_culture_way_of_life',
                       df_PDS.columns[11]:'Q5_d_PG_learn_history_host_country',
                       df_PDS.columns[12]:'Q5_e_PG_expl_heritage',
                       df_PDS.columns[13]:'Q5_f_PG_better_know_regional_polit_curnt_evnts_',
                       df_PDS.columns[14]:'Q5_g_PG_learned_about_myself',
                       df_PDS.columns[15]:'Q5_h_PG_earnd_credt_to_grad',
                       df_PDS.columns[16]:'Q5_i_PG_gained_independ',
                       df_PDS.columns[17]:'Q5_j_PG_expl_environ',
                       df_PDS.columns[18]:'Q5_k_PG_positiv_contribut_host_commun',
                       df_PDS.columns[19]:'Q5_l_PG_qual_other', 
                       df_PDS.columns[20]:'Q15_a_DM_ID_Q1',
                       df_PDS.columns[21]:'Q15_b_DM_HI_Q1',
                       df_PDS.columns[22]:'Q15_c_DM_SP_Q1',
                       df_PDS.columns[23]:'Q15_d_DM_PF_Q1',
                       df_PDS.columns[24]:'Q15_e_DM_EV_Q1',
                       df_PDS.columns[25]:'Q16_a_DM_ID_Q2',
                       df_PDS.columns[26]:'Q16_b_DM_HI_Q2',
                       df_PDS.columns[27]:'Q16_c_DM_SP_Q2',
                       df_PDS.columns[28]:'Q16_d_DM_PF_Q2',
                       df_PDS.columns[29]:'Q16_e_DM_EV_Q2',
                       df_PDS.columns[30]:'Q17_a_DM_ID_Q3',
                       df_PDS.columns[31]:'Q17_b_DM_HI_Q3',
                       df_PDS.columns[32]:'Q17_c_DM_SP_Q3',
                       df_PDS.columns[33]:'Q17_d_DM_PF_Q3',
                       df_PDS.columns[34]:'Q17_e_DM_EV_Q3',   
                       
}
                     , inplace=True)

In [ ]:
#drop second row of data for both surveys, surveymonkey exports csv files with two row headers
df_PPS.drop([0], axis=0, inplace=True)
df_PDS.drop([0], axis=0, inplace=True)

In [ ]:
#lets take a look at what is left of the pre-departure survey questions-
df_PDS.info()

In [ ]:
#We will needed to convert PDS numeric columns to dates and ints using "to_datetime" and "to_numeric", that is because
#when importing the file, these numeric columns were encoded as strings. 
df_PDS.resp_start_date = pd.to_datetime(df_PDS.resp_start_date)
df_PDS.resp_end_date = pd.to_datetime(df_PDS.resp_end_date)
df_PDS.Q1_would_rec = pd.to_numeric(df_PDS['Q1_would_rec'], errors='ignore')
df_PDS.Q15_a_DM_ID_Q1 = pd.to_numeric(df_PDS['Q15_a_DM_ID_Q1'], errors='ignore')
df_PDS.Q15_b_DM_HI_Q1 = pd.to_numeric(df_PDS['Q15_b_DM_HI_Q1'], errors='ignore')
df_PDS.Q15_c_DM_SP_Q1 = pd.to_numeric(df_PDS['Q15_c_DM_SP_Q1'], errors='ignore')
df_PDS.Q15_d_DM_PF_Q1 = pd.to_numeric(df_PDS['Q15_d_DM_PF_Q1'], errors='ignore')
df_PDS.Q15_e_DM_EV_Q1 = pd.to_numeric(df_PDS['Q15_e_DM_EV_Q1'], errors='ignore')
df_PDS.Q16_a_DM_ID_Q2 = pd.to_numeric(df_PDS['Q16_a_DM_ID_Q2'], errors='ignore')
df_PDS.Q16_b_DM_HI_Q2 = pd.to_numeric(df_PDS['Q16_b_DM_HI_Q2'], errors='ignore')
df_PDS.Q16_c_DM_SP_Q2 = pd.to_numeric(df_PDS['Q16_c_DM_SP_Q2'], errors='ignore')
df_PDS.Q16_d_DM_PF_Q2 = pd.to_numeric(df_PDS['Q16_d_DM_PF_Q2'], errors='ignore')
df_PDS.Q16_e_DM_EV_Q2 = pd.to_numeric(df_PDS['Q16_e_DM_EV_Q2'], errors='ignore')
df_PDS.Q17_a_DM_ID_Q3 = pd.to_numeric(df_PDS['Q17_a_DM_ID_Q3'], errors='ignore')
df_PDS.Q17_b_DM_HI_Q3 = pd.to_numeric(df_PDS['Q17_b_DM_HI_Q3'], errors='ignore')
df_PDS.Q17_c_DM_SP_Q3 = pd.to_numeric(df_PDS['Q17_c_DM_SP_Q3'], errors='ignore')
df_PDS.Q17_d_DM_PF_Q3 = pd.to_numeric(df_PDS['Q17_d_DM_PF_Q3'], errors='ignore')
df_PDS.Q17_e_DM_EV_Q3 = pd.to_numeric(df_PDS['Q17_e_DM_EV_Q3'], errors='ignore')


In [ ]:
#Carry out the same step for the PPS - convert numeric columns to dates and ints
df_PPS.resp_start_date = pd.to_datetime(df_PPS.resp_start_date)
df_PPS.resp_end_date = pd.to_datetime(df_PPS.resp_end_date)
df_PPS.Q1_would_rec = pd.to_numeric(df_PPS['Q1_would_rec'], errors='ignore')
df_PPS.Q32_a_DM_ID_Q1 = pd.to_numeric(df_PPS['Q32_a_DM_ID_Q1'], errors='ignore')
df_PPS.Q32_b_DM_HI_Q1 = pd.to_numeric(df_PPS['Q32_b_DM_HI_Q1'], errors='ignore')
df_PPS.Q32_c_DM_SP_Q1 = pd.to_numeric(df_PPS['Q32_c_DM_SP_Q1'], errors='ignore')
df_PPS.Q32_d_DM_PF_Q1 = pd.to_numeric(df_PPS['Q32_d_DM_PF_Q1'], errors='ignore')
df_PPS.Q32_e_DM_EV_Q1 = pd.to_numeric(df_PPS['Q32_e_DM_EV_Q1'], errors='ignore')
df_PPS.Q33_a_DM_ID_Q2 = pd.to_numeric(df_PPS['Q33_a_DM_ID_Q2'], errors='ignore')
df_PPS.Q33_b_DM_HI_Q2 = pd.to_numeric(df_PPS['Q33_b_DM_HI_Q2'], errors='ignore')
df_PPS.Q33_c_DM_SP_Q2 = pd.to_numeric(df_PPS['Q33_c_DM_SP_Q2'], errors='ignore')
df_PPS.Q33_d_DM_PF_Q2 = pd.to_numeric(df_PPS['Q33_d_DM_PF_Q2'], errors='ignore')
df_PPS.Q33_e_DM_EV_Q2 = pd.to_numeric(df_PPS['Q33_e_DM_EV_Q2'], errors='ignore')
df_PPS.Q34_a_DM_ID_Q3 = pd.to_numeric(df_PPS['Q34_a_DM_ID_Q3'], errors='ignore')
df_PPS.Q34_b_DM_HI_Q3 = pd.to_numeric(df_PPS['Q34_b_DM_HI_Q3'], errors='ignore')
df_PPS.Q34_c_DM_SP_Q3 = pd.to_numeric(df_PPS['Q34_c_DM_SP_Q3'], errors='ignore')
df_PPS.Q34_d_DM_PF_Q3 = pd.to_numeric(df_PPS['Q34_d_DM_PF_Q3'], errors='ignore')
df_PPS.Q34_e_DM_EV_Q3 = pd.to_numeric(df_PPS['Q34_e_DM_EV_Q3'], errors='ignore')



In [ ]:
df_PPS.shape

In [ ]:
df_PDS.shape

In [ ]:
#drop duplicate rows from PPS, PDS as for some reason the administrators of the survey sent created multiple
#collectors for the same set of student groups. This was done when we first switched protocol, from
#weblink collectors links to email collectors. With data LEFT joined, any duplicates will multiply records and 
#skew numbers higher. 

df_PPS.drop_duplicates(subset=['email', 'program_code']
                       , inplace=True)
df_PDS.drop_duplicates(subset=['email', 'program_code']
                       , inplace=True)

In [ ]:
df_PPS.shape

In [ ]:
df_PDS.shape

In [ ]:
#Lets import data from our CRM, normally pulled and shaped using SQL, when connected to a normal database. For
#our purposes, the data is present in a CSV file. It will be important to mask this data for public consumption
#as this was taken from an office's (SAO) actual CRM. 
df_CRM = pd.read_csv('CRM_data.csv')

In [ ]:
#in order to mask the SAO's personid, we are going to create a masked personid column by randomly generating numbers
#into a new dataframe, based on the shape of the original personid column (length)
#This is also why we imported the random function at the beginning of the notebook. 

data = np.random.randint(100000,999999,df_CRM.shape[0])
df_rand = pd.DataFrame(data, columns=['maskPersonid'])


In [ ]:
#Then we join that new, random numbered column to the dataframe, and we will eventually drop the original personid 
#column
numbers = df_rand['maskPersonid']
df_CRM = df_CRM.join(numbers)

In [ ]:
#In order to mask the data even more, lets ramdomize the cities using a list of cities that students could potentially 
#travel to, then we need to give these city their appropriate countries so the Tableau map (using the next line of 
#code)function has the correct data to plot coordinates



citieslist = ["Newcastle", "Barcelona","Salamanca","Gold Coast","Port Elizabeth","Prague","Dublin","Granada","Berlin","Florence","Seoul","Athens","Auckland","Paris","Christchurch",
"Busan","Madrid","Galway","Valpara√≠so and Vi√±a del Mar","Barranquilla","London","Cape Town","San Jos√©","Wellington","Melbourne","Seville","Cusco","Canberra","Shanghai",
"Florian√≥polis","Rome","Reading","Bangkok","Santiago","Lima","Heredia","Buenos Aires","Sydney","Bilbao","Limerick","Santander","Stirling","Adelaide","Dunedin","Glasgow",
"Amsterdam","Meknes","Ho Chi Minh City","M√°laga","Lille","Brisbane","Milan","Suva","Townsville","Palmerston North","Medell√≠n","Tokyo","Oxford","Cairns","Perth","Cartago",
"Valencia"]

df_CRM['city'] = np.random.choice(citieslist, len(df_CRM), replace=True)


In [ ]:
#Set random cities to real countries using .loc-

df_CRM.loc[df_CRM['city'] == 'Newcastle', 'Country'] = 'Australia'
df_CRM.loc[df_CRM['city'] == 'Barcelona', 'Country'] = 'Spain'
df_CRM.loc[df_CRM['city'] == 'Salamanca', 'Country'] = 'Spain'
df_CRM.loc[df_CRM['city'] == 'Gold Coast', 'Country'] = 'Australia'
df_CRM.loc[df_CRM['city'] == 'Port Elizabeth', 'Country'] = 'South Africa'
df_CRM.loc[df_CRM['city'] == 'Prague', 'Country'] = 'Czech Republic'
df_CRM.loc[df_CRM['city'] == 'Dublin', 'Country'] = 'Ireland'
df_CRM.loc[df_CRM['city'] == 'Granada', 'Country'] = 'Spain'
df_CRM.loc[df_CRM['city'] == 'Berlin', 'Country'] = 'Germany'
df_CRM.loc[df_CRM['city'] == 'Florence', 'Country'] = 'Italy'
df_CRM.loc[df_CRM['city'] == 'Seoul', 'Country'] = 'South Korea'
df_CRM.loc[df_CRM['city'] == 'Athens', 'Country'] = 'Greece'
df_CRM.loc[df_CRM['city'] == 'Auckland', 'Country'] = 'New Zealand'
df_CRM.loc[df_CRM['city'] == 'Paris', 'Country'] = 'France'
df_CRM.loc[df_CRM['city'] == 'Christchurch', 'Country'] = 'New Zealand'
df_CRM.loc[df_CRM['city'] == 'Busan', 'Country'] = 'South Korea'
df_CRM.loc[df_CRM['city'] == 'Madrid', 'Country'] = 'Spain'
df_CRM.loc[df_CRM['city'] == 'Galway', 'Country'] = 'Ireland'
df_CRM.loc[df_CRM['city'] == 'Valpara√≠so and Vi√±a del Mar', 'Country'] = 'Chile'
df_CRM.loc[df_CRM['city'] == 'Barranquilla', 'Country'] = 'Colombia'
df_CRM.loc[df_CRM['city'] == 'London', 'Country'] = 'England'
df_CRM.loc[df_CRM['city'] == 'Cape Town', 'Country'] = 'South Africa'
df_CRM.loc[df_CRM['city'] == 'San Jos√©', 'Country'] = 'Costa Rica'
df_CRM.loc[df_CRM['city'] == 'Wellington', 'Country'] = 'New Zealand'
df_CRM.loc[df_CRM['city'] == 'Seville', 'Country'] = 'Spain'
df_CRM.loc[df_CRM['city'] == 'Cusco', 'Country'] = 'Peru'
df_CRM.loc[df_CRM['city'] == 'Canberra', 'Country'] = 'Australia'
df_CRM.loc[df_CRM['city'] == 'Shanghai', 'Country'] = 'China'
df_CRM.loc[df_CRM['city'] == 'Florian√≥polis', 'Country'] = 'Brazil'
df_CRM.loc[df_CRM['city'] == 'Rome', 'Country'] = 'Italy'
df_CRM.loc[df_CRM['city'] == 'Reading', 'Country'] = 'England'
df_CRM.loc[df_CRM['city'] == 'Bangkok', 'Country'] = 'Thailand'
df_CRM.loc[df_CRM['city'] == 'Santiago', 'Country'] = 'Chile'
df_CRM.loc[df_CRM['city'] == 'Lima', 'Country'] = 'Peru'
df_CRM.loc[df_CRM['city'] == 'Heredia', 'Country'] = 'Costa Rica'
df_CRM.loc[df_CRM['city'] == 'Buenos Aires', 'Country'] = 'Argentina'
df_CRM.loc[df_CRM['city'] == 'Sydney', 'Country'] = 'Australia'
df_CRM.loc[df_CRM['city'] == 'Bilbao', 'Country'] = 'Spain'
df_CRM.loc[df_CRM['city'] == 'Limerick', 'Country'] = 'Ireland'
df_CRM.loc[df_CRM['city'] == 'Stirling', 'Country'] = 'Scotland'
df_CRM.loc[df_CRM['city'] == 'Adelaide', 'Country'] = 'Australia'
df_CRM.loc[df_CRM['city'] == 'Dunedin', 'Country'] = 'New Zealand'
df_CRM.loc[df_CRM['city'] == 'Glasgow', 'Country'] = 'Scotland'
df_CRM.loc[df_CRM['city'] == 'Amsterdam', 'Country'] = 'The Netherlands'
df_CRM.loc[df_CRM['city'] == 'Meknes', 'Country'] = 'Morocco'
df_CRM.loc[df_CRM['city'] == 'Ho Chi Minh City', 'Country'] = 'Vietnam'
df_CRM.loc[df_CRM['city'] == 'M√°laga', 'Country'] = 'Spain'
df_CRM.loc[df_CRM['city'] == 'Lille', 'Country'] = 'France'
df_CRM.loc[df_CRM['city'] == 'Brisbane', 'Country'] = 'Australia'
df_CRM.loc[df_CRM['city'] == 'Milan', 'Country'] = 'Italy'
df_CRM.loc[df_CRM['city'] == 'Suva', 'Country'] = 'Fiji'
df_CRM.loc[df_CRM['city'] == 'Townsville', 'Country'] = 'Australia'
df_CRM.loc[df_CRM['city'] == 'Palmerston North', 'Country'] = 'New Zealand'
df_CRM.loc[df_CRM['city'] == 'Medell√≠n', 'Country'] = 'Colombia'
df_CRM.loc[df_CRM['city'] == 'Tokyo', 'Country'] = 'Japan'
df_CRM.loc[df_CRM['city'] == 'Oxford', 'Country'] = 'England'
df_CRM.loc[df_CRM['city'] == 'Perth', 'Country'] = 'Australia'
df_CRM.loc[df_CRM['city'] == 'Cartago', 'Country'] = 'Costa Rica'
df_CRM.loc[df_CRM['city'] == 'Valencia', 'Country'] = 'Spain'
df_CRM.loc[df_CRM['city'] == 'Melbourne', 'Country'] = 'Australia'
df_CRM.loc[df_CRM['city'] == 'Cairns', 'Country'] = 'Australia'
df_CRM.loc[df_CRM['city'] == 'Santander', 'Country'] = 'Spain'

In [ ]:
#Lets take a random sample of this new, modified masked dataframe so we can better mask the data, this sample is 
#between 50% and 90% of the of the true dataset, we won't know which because we are using the random.uniform function-
x = random.uniform(.5, .9)

df_CRM = df_CRM.sample(frac=(x))


#because we are going to be left joining the survey data to this set, we don't want to take a random fraction of the
#survey data. We are going to be joining on email_address and program_code. 

In [ ]:
#Lets drop all of the columns we don't need, of course it would be interesting to analyze learning and satifaction
#with referral type, method, and place of birth, etc.. but for this analysis lets keep it simple and only keep 
#the columns we are interested in. This CRM data was prevously cleaned using SQL - for example and GPA = 0 data 
#was converted to N/A, duplicates were removed, and records are based on session id, not package (account) or 
#unique person ID
df_CRM.drop(df_CRM.columns[39:45], axis=1, inplace = True)
df_CRM.drop(df_CRM.columns[33:35], axis=1, inplace = True)
df_CRM.drop(df_CRM.columns[30:32], axis=1, inplace = True)
df_CRM.drop(df_CRM.columns[26], axis=1, inplace = True)
df_CRM.drop(df_CRM.columns[23], axis=1, inplace = True)
df_CRM.drop(df_CRM.columns[17:20], axis=1, inplace = True)
df_CRM.drop(df_CRM.columns[10:16], axis=1, inplace = True)
df_CRM.drop(df_CRM.columns[5:8], axis=1, inplace = True)
df_CRM.drop(df_CRM.columns[2:4], axis=1, inplace = True)
df_CRM.drop(df_CRM.columns[0], axis=1, inplace = True)

In [ ]:
#change the start date and end date of programs to a datetime datatype 
df_CRM.enddate = pd.to_datetime(df_CRM.enddate)
df_CRM.startdate = pd.to_datetime(df_CRM.startdate)

In [ ]:
#Lets see what our CRM data looks like now-
df_CRM.info()

In [ ]:
#we also need to create a new business division, one not in our current CRM, based on the program_code, using 
#the first character in the code = i, and the 4th character = T or G
df_CRM['SAOSP1'] = df_CRM['program_code'].str[0]
df_CRM['SAOSP2'] = df_CRM['program_code'].str[3]

In [ ]:
#create another dataframe containing records where all 
#rows with first letter in program code = I and forth letter 
#equals T or G, lets call this dataframe "df_PN"
df_PN = df_CRM.loc[(df_CRM['SAOSP1'] == 'I') 
        & ((df_CRM['SAOSP2'] == 'T') | (df_CRM['SAOSP2'] == 'G'))]

In [ ]:
#This formula removes PN dataframe from our current CRM dataframe
#just taking it out and creating a "temporary" subframe
df_CRM = pd.concat([df_CRM, df_PN, df_PN]).drop_duplicates(keep=False)

In [ ]:
#replace the SAO business division "bus div" with the correct Business
#division, using "SP" as the acronymn for the business division not currently in our CRM
#remember we only do this to the df_PN "temporary" subframe-
df_PN.bus_div.replace(to_replace=['SAO'], value='SP', inplace=True)

In [ ]:
#combine df_PN database with df_CRM then drop both SAOSP1 and SAOSP2 columns (next line of code) 
#to bring our Dataframe back together
df_CRM = pd.concat([df_CRM, df_PN], ignore_index=True)

In [ ]:
#This code drops the two additional columns we created in order to make the additional business division
df_CRM.drop(df_CRM.columns[21:23], axis=1, inplace=True)

In [ ]:
#replace out city names with correct names, removing symbols inherited from data pull/csv
#this will ease transition into Tableau
df_CRM.city.replace(to_replace=['M√°laga'],value='Málaga', inplace=True)
df_CRM.city.replace(to_replace=['Florian√≥polis'],value='Florianópolis', inplace=True)
df_CRM.city.replace(to_replace=['San Jos√©'],value='San José', inplace=True)
df_CRM.city.replace(to_replace=['San Jos?'],value='San José', inplace=True)
df_CRM.city.replace(to_replace=['Medell√≠n'],value='Medellín', inplace=True)
df_CRM.city.replace(to_replace=['Valpara√≠so and Vi√±a del Mar'],value='Valparaíso and Viña del Mar', inplace=True)

In [ ]:
#Lets modify the dataframe to only include students who participated in programs in 2018-2019, and had their account 
#status as completed or currently abroad. 
df_CRM = df_CRM[df_CRM['account_status'].isin(['Completed', 'Currently Abroad']) ]
df_CRM = df_CRM[df_CRM['year'].isin([2018,2019]) ]



In [ ]:
#Now that I have my dataframes created, this will allow me to combine all surveys and CRM data to same excel file,
#and I can join my data in Tableau, using two fields - email_address and program_code
LMS = 'LMS'+'.xlsx'
LMS

In [ ]:
#Gives final excell document with all tabs in correct order, REMEMBER TO RENAME WITH TODAY'S DATE!
writer = pd.ExcelWriter(LMS, engine='xlsxwriter')

df_CRM.to_excel(writer, sheet_name='CRM', index=False)

df_PDS.to_excel(writer, sheet_name='PDS', index=False)

df_PPS.to_excel(writer, sheet_name='PPS', index=False)

writer.save()
                            